# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Building composable functions”

Abraham warns the reader that this lesson represents a rapid departure from the comfort zone of C#. This is done by identifying several F# equivalents of what we might be used to in C#:

- The F# equivalent of C# method overloading is _currying_.
- The F# equivalent of C# extension methods is _pipelining_ with the forward-pipe operator, `|>`.
- The F# way to express what C# folks might call imperative method chaining is with the compose operator, `>>`.


## Comparing methods and functions

| | C# methods | F# `let`-bound functions |
|- |- |-
| Behavior | Statements of expressions | Expressions by default |
| Scope | Instance [object] or static [type] | Static [module level of nested function] |
| Overloading | Allowed | Not supported |
| Currying | Not supported | Native support |

Just because C# methods do not support currying does not mean it is not supported in the language. `Func` in C# [can be used](https://wensveen.wordpress.com/2010/03/01/currying-in-csharp/).

In the world of F#, currying can be thought of as a extremely constrained approach to method overloading, limited to one parameter per “overload.”


## Partial function application

Partial function application is a way to avoid the use of the term [_currying_](https://en.wikipedia.org/wiki/Currying). This might be done because currying refers to a person, [Haskell Brooks Curry](https://en.wikipedia.org/wiki/Haskell_Curry).

>Partially applied functions are one of the most powerful parts of the function system in F# as compared to C#. These functions open up all sorts of interesting possibilities …the following two functions appear to do the same thing, except that one uses brackets (parentheses) and commas for input arguments (like C#), and one doesn’t. The former is referred to as _tupled_ form and the latter as _curried_ form.


In [ ]:
let tupledAdd(a,b) = a + b

tupledAdd (5,10)

15

In [ ]:
let curriedAdd a b = a + b

curriedAdd 5 10

15

@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.245901+0512fe0a05f5a99419a543463ee68233a3c60371Build date: 2021-09-10T01:15:17.7422137Zhttps://github.com/dotnet/interactive
